In [0]:
# ---------------------------------------------------------------------------- #
# An implementation of https://arxiv.org/pdf/1512.03385.pdf                    #
# See section 4.2 for the model architecture on CIFAR-10                       #
# Some part of the code was referenced from below                              #
# https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py   #
# ---------------------------------------------------------------------------- #

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [0]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

In [0]:
# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train the model
def train(model):
  total_step = len(train_loader)
  curr_lr = learning_rate
  for epoch in range(num_epochs):
      for i, (images, labels) in enumerate(train_loader):
          images = images.to(device)
          labels = labels.to(device)

          # Forward pass
          outputs = model(images)
          loss = criterion(outputs, labels)

          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          if (i+1) % 100 == 0:
              print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                     .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

      # Decay learning rate
      if (epoch+1) % 20 == 0:
          curr_lr /= 3
          update_lr(optimizer, curr_lr)

# Test the model
def test(model):
  model.eval()
  with torch.no_grad():
      correct = 0
      total = 0
      for images, labels in test_loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
# torch.save(model.state_dict(), 'resnet.ckpt')

In [0]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=True)

# 3. add a scalar multiplier (initialized at 1) in every branch 
# mult layer
# https://discuss.pytorch.org/t/is-scale-layer-available-in-pytorch/7954/8
class ScaleLayer(nn.Module):

  def __init__(self, init_value=1):
    super().__init__()
    self.scale = nn.Parameter(torch.FloatTensor([init_value]))

  def forward(self, input):
       return input * self.scale
    
# bias layer
class BiasLayer(nn.Module):

  def __init__(self, init_value=0):
    super().__init__()
    self.bias = nn.Parameter(torch.FloatTensor([init_value]))

  def forward(self, input):
       return input + self.bias

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.bias1 = BiasLayer()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
#         self.bn1 = nn.BatchNorm2d(out_channels)
        self.bias2 = BiasLayer()
        self.relu = nn.ReLU(inplace=True)
        self.bias3 = BiasLayer()
        self.conv2 = conv3x3(out_channels, out_channels)
    # 3. add a scalar multiplier (initialized at 1) in every branch 
        self.mult = ScaleLayer()
        self.bias4 = BiasLayer()
#         self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
  
  
        # 1. initialize the last layer of each residual branch to 0
        if self.downsample:  
          torch.nn.init.constant_(self.downsample.weight, 0)
          torch.nn.init.constant_(self.downsample.bias, 0)
        else:
          torch.nn.init.constant_(self.conv2.weight, 0)
          torch.nn.init.constant_(self.conv2.bias, 0)
      # bias=False in conv3x3 so this doesn't work NOTE FOR LATER TO TRY TO SET TRUE
      # SET BIAS TO TRUE
#         
      
        # 2. initialize every other layer using a standard method (e.g., He et al. (2015))
        # NOTE FOR LATER TRY FAN_OUT
      # comment this out because then initialize wight layers inside residual branches lol
#         torch.nn.init.kaiming_normal_(self.conv1.weight, mode='fan_in', nonlinearity='leaky_relu')
#         torch.nn.init.kaiming_normal_(self.conv1.bias, mode='fan_in', nonlinearity='leaky_relu')
        
        # 2. L - number of residual branches; m - a small number, e.g. 2 or 3
        # scale only the weight layers inside residual branches by L^(−1)/2m−2,
        # i.e. can scale by sqrt(L)
        scale_factor = torch.sqrt(torch.tensor(num_layers*3, dtype=torch.float64))
        self.conv1.weight.data.mul_(scale_factor)
        
  
        
    def forward(self, x):
        residual = x
        # 3. add a scalar bias (initialized at 0) before each convolution, linear, and element-wise activation layer
        out = self.bias1(x)
        out = self.conv1(out)
#         out = self.bn1(out)
        out = self.bias2(out)
        out = self.relu(out)
        out = self.bias3(out)
        out = self.conv2(out)
    # 3. add a scalar multiplier (initialized at 1) in every branch 
        out = self.mult(out)
        out = self.bias4(out)
#         out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
#         self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
        # 1. initialize the classification layer to 0
        torch.nn.init.constant_(self.fc.weight, 0)
        torch.nn.init.constant_(self.fc.bias, 0)
        
        # 2. initialize every other layer using a standard method (e.g., He et al. (2015))
        # NOTE FOR LATER TRY FAN_OUT
        torch.nn.init.kaiming_normal_(self.conv.weight, mode='fan_in', nonlinearity='leaky_relu')
#         torch.nn.init.kaiming_normal_(self.conv.bias, mode='fan_in', nonlinearity='leaky_relu')
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = conv3x3(self.in_channels, out_channels, stride=stride)
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
#         out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [0]:
# Hyper-parameters
num_layers = 2
num_epochs = 1
learning_rate = 0.001

In [0]:
model = ResNet(ResidualBlock, [num_layers, num_layers, num_layers]).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
# print(list(model.named_modules())[0])
# print(list(model.named_modules())[-3][1].bias)

In [0]:
train(model)
test(model)

Epoch [1/1], Step [100/500] Loss: 2.0773
Epoch [1/1], Step [200/500] Loss: 1.8661
Epoch [1/1], Step [300/500] Loss: 1.9114
Epoch [1/1], Step [400/500] Loss: 1.7057
Epoch [1/1], Step [500/500] Loss: 1.6334
Accuracy of the model on the test images: 37.36 %
